In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from math import sqrt


train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
train_df.drop(['ID','Age', 'Location','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher'],axis = 1, inplace = True)
test_df.drop(['ID','Age', 'Location','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher'],axis = 1, inplace = True)
submission_df = pd.read_csv('./data/sample_submission.csv')

In [2]:
user_dict = {user_id : i for i, user_id in enumerate(train_df['User-ID'].unique())}
book_dict = {book_id : i for i, book_id in enumerate(train_df['Book-ID'].unique())}

In [3]:
train_df['User-ID'] = train_df['User-ID'].map(user_dict)
train_df['Book-ID'] = train_df['Book-ID'].map(book_dict)

test_df['User-ID'] = test_df['User-ID'].map(user_dict)
test_df['Book-ID'] = test_df['Book-ID'].map(book_dict)

In [4]:
class RecommenderModel(nn.Module):
    def __init__(self, num_users, num_books, embedding_size = 50):
        super(RecommenderModel, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_size)
        self.book_embeddings = nn.Embedding(num_books, embedding_size)
        self.fc1 = nn.Linear(embedding_size * 2, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        
    def forward(self, user, book):
        user_embed = self.user_embeddings(user)
        book_embed = self.book_embeddings(book)
        x = torch.cat([user_embed, book_embed], dim = 1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = RecommenderModel(num_users = len(user_dict), num_books = len(book_dict)).cuda()

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fn = torch.nn.MSELoss()

In [6]:
# train model

from tqdm import tqdm, tqdm_notebook

num_epochs = 30
batch_size = 128
for epoch in tqdm(range(num_epochs)):
    train_losses = []
    for i in range(0, len(train_df), batch_size):
        batch = train_df[i:i+batch_size]
        user = torch.tensor(batch['User-ID'].values, dtype = torch.long).cuda()
        book = torch.tensor(batch['Book-ID'].values, dtype = torch.long).cuda()
        rating = torch.tensor(batch['Book-Rating'].values, dtype = torch.float32).cuda()
        
        optimizer.zero_grad()
        output = model(user, book)
        loss = loss_fn(output.view(-1), rating)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
    print('Epoch : {} Train Loss : {:.4f}'.format(epoch+1, sum(train_losses)/len(train_losses)))

  3%|▎         | 1/30 [00:20<10:00, 20.70s/it]

Epoch : 1 Train Loss : 14.8123


  7%|▋         | 2/30 [00:40<09:26, 20.24s/it]

Epoch : 2 Train Loss : 13.8204


 10%|█         | 3/30 [01:00<08:56, 19.86s/it]

...

Epoch : 28 Train Loss : 2.3637


 97%|█████████▋| 29/30 [09:25<00:19, 19.44s/it]

Epoch : 29 Train Loss : 2.3108


100%|██████████| 30/30 [09:44<00:00, 19.50s/it]

Epoch : 30 Train Loss : 2.2214


In [7]:
with torch.no_grad():
    user = torch.tensor(test_df['User-ID'].values, dtype = torch.long).cuda()
    book = torch.tensor(test_df['Book-ID'].values, dtype = torch.long).cuda()
    output = model(user, book)
    predicted_ratings = output.cpu().view(-1).tolist()

In [8]:
submission_df['Book-Rating'] = predicted_ratings
submission_df.to_csv('submission2.csv',index = False)